In [ ]:
using Pkg

# Activate local project environment
Pkg.activate(expanduser("/g/data/v46/txs156/ocean-ensembles/analysis/"))

# Add the global environment to the package search path
push!(LOAD_PATH, expanduser("/g/data/v46/txs156/ocean-ensembles/.julia/environments/v1.10/"))

# Pkg.precompile()

using CairoMakie, IJulia, JLD2  # From global
using Oceananigans  # From local

┌ Warning: Error requiring `IntervalArithmetic` from `TaylorSeries`
│   exception =
│    LoadError: UndefVarError: `IntervalBox` not defined
│    Stacktrace:
│      [1] top-level scope
│        @ /g/data/e14/txs156/.julia/packages/TaylorSeries/ZBRjU/src/intervals.jl:171
│      [2] include(mod::Module, _path::String)
│        @ Base ./Base.jl:495
│      [3] include(x::String)
│        @ TaylorSeries /g/data/e14/txs156/.julia/packages/TaylorSeries/ZBRjU/src/TaylorSeries.jl:17
│      [4] top-level scope
│        @ /g/data/e14/txs156/.julia/packages/Requires/1eCOK/src/Requires.jl:40
│      [5] eval
│        @ ./boot.jl:385 [inlined]
│      [6] eval
│        @ /g/data/e14/txs156/.julia/packages/TaylorSeries/ZBRjU/src/TaylorSeries.jl:17 [inlined]
│      [7] (::TaylorSeries.var"#54#57")()
│        @ TaylorSeries /g/data/e14/txs156/.julia/packages/Requires/1eCOK/src/require.jl:101
│      [8] macro expansion
│        @ timing.jl:395 [inlined]
│      [9] err(f::Any, listener::Module, modname::St

In [ ]:
experiment_path = expanduser("/g/data/v46/txs156/ocean-ensembles/experiments/")
figdir = expanduser("/g/data/v46/txs156/ocean-ensembles/figures/")
output_path = expanduser("/g/data/v46/txs156/ocean-ensembles/outputs/")

In [ ]:
variables_basins = ["_global_", "_atl_", "_ipac_"]
variables_diags = ["zonal", "depth", "tot"]
variables_tracers = ["T", "S", "dV"]
variables_velocities =  ["u", "v", "w"]
variables_vel_volumes = ["dV_u", "dV_v", "dV_w"]
# variables_fluxes = ["latent_heat", "sensible_heat", "water_vapor", "x_momentum", "y_momentum"]

tracercontent_vars = vec([tracer * basin * diag for tracer in variables_tracers,
                         basin in variables_basins,
                         diag in variables_diags])

transport_vars = vcat(
    vec([velocity * basin * diag for velocity in variables_velocities,
                                 basin in variables_basins,
                                 diag in variables_diags]),
    vec([volume * basin * diag for volume in variables_vel_volumes,
                               basin in variables_basins,
                               diag in variables_diags])
)

function create_dict(vars, path)
    dicts = Dict()
    for var in vars
        try
            # Surface
            dicts[var] = FieldTimeSeries(path, var)
        catch e
            if e isa KeyError
                @warn "Skipping variable $var: Key not found in file."
            else
                rethrow(e)
            end
        end
    end
    return dicts
end

@info "I am loading the otc" 
OTC = create_dict(tracercontent_vars, output_path * "ocean_tracer_content_onedeg_iteration0_rank0.jld2")

@info "I am loading the mass transport" 
masstrans = create_dict(transport_vars, output_path * "mass_transport_onedeg_iteration0_rank0.jld2")

# @info "I am loading the fluxes" 
# fluxes = create_dict(variables_fluxes, output_path * "fluxes_RYF1deg.jld2")

@info "I am loading the surface" 
surface = create_dict(vcat(variables_tracers, variables_velocities), output_path * "global_surface_fields_onedeg_iteration0_rank0.jld2")


In [ ]:
temp_too_high = 35
times = surface["T"].times
mxtemp = zeros(length(times))  # Ensure the array is 1D with the correct size
mntemp = zeros(length(times))  # Ensure the array is 1D with the correct size
x_indices = []
y_indices = []
hightmp_counter = zeros(length(times))
for time_step in eachindex(times)  # Proper way to iterate over indices
    mxtemp[time_step] = maximum(filter(!isnan, surface["T"][time_step]))  # Ignore NaNs
    mntemp[time_step] = minimum(filter(!isnan, surface["T"][time_step]))  # Ignore NaNs
    T_array = interior(surface["T"][time_step])
    hightmp_counter[time_step] = count(T_array .> temp_too_high)
    # Extract the interior data
    interior_data = interior(surface["T"][time_step])

    # Find the (i, j) index of the maximum temperature in the interior
    max_index = argmax(interior_data)

    if mxtemp[time_step] >=temp_too_high
        push!(x_indices, max_index[1])
        push!(y_indices, max_index[2])
    end
end


In [ ]:


fig = Figure(size = (1600, 1200))
ax1 = Axis(fig[1, 1], xlabel="Years", ylabel="Temperature", title="Max Surface\n Temperature Over Time")
ax2 = Axis(fig[1,2], xlabel="Years", ylabel="Number of cells", title="# of surface\n grid cells above 40C")
ax3 = Axis(fig[2:3,:], xlabel="Longitude Index", ylabel="Latitude Index", title="Location of surface\n grid cells above 40C")

lines!(ax1, times/(3600*24*365.25), mxtemp)
# lines!(ax1, times/(3600*24*365.25), mntemp)
lines!(ax2, times/(3600*24*365.25), (hightmp_counter))
hm = heatmap!(ax3, dropdims(mean(interior(surface["T"]), dims = 4), dims = (3,4)), colorrange = (-2,35), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[4, :], hm, label = "Temperature (ᵒC)", vertical = false)

scatter!(ax3, x_indices, y_indices, color=:black, markersize=20)

xlims!(ax2, 0,1.5)
xlims!(ax1, 0,1.5)

save(figdir*"minmax_diff_quarter.png", fig)

display(fig)

In [ ]:
using NCDatasets

ERA5_hfds = expanduser("/Users/tsohail/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Desktop/Desktop - Taimoor’s UNSW MacBook Pro/UNSW Work/ERA5/surf_HF_ERA5.nc")
ERA5_lm = expanduser("/Users/tsohail/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Desktop/Desktop - Taimoor’s UNSW MacBook Pro/UNSW Work/ERA5/ERA5_land_mask.nc")

JRA_fluxes_sensible = Dataset(expanduser("/Users/tsohail/OneDrive - The University of Melbourne/uom/Data/sensible199001_199012.nc"), "r")
JRA_fluxes_latent = Dataset(expanduser("/Users/tsohail/OneDrive - The University of Melbourne/uom/Data/latent199001_199012.nc"), "r")

JRA_landmask = Dataset(expanduser("/Users/tsohail/OneDrive - The University of Melbourne/uom/Data/JRA_land.nc"), "r")
JRA_area = Dataset(expanduser("/Users/tsohail/OneDrive - The University of Melbourne/uom/Data/JRA_area.nc"), "r")

ds_hfds = Dataset(ERA5_hfds, "r")
ds_lm = Dataset(ERA5_lm, "r")

degrees2kilometers = 111.19492664455873
lon_deg_dist = cos.(abs.(ds_hfds["latitude"][:]) .* (2*pi/360)) .* 111321 .* degrees2kilometers .* 1000 .* 0.25^2

d_area_ERA5 = repeat(lon_deg_dist, 1, length(ds_hfds["longitude"][:]))

In [ ]:
land_mask = (1 .-repeat(Array(JRA_landmask["var81"]), 1, 1, 12))
area_mask = repeat(Array(JRA_area["areacella"]), 1, 1, 12)

JRA55_latentheat_90 = Array(JRA_fluxes_latent["var121"]).*land_mask.*area_mask #units: W
JRA55_sensibleheat_90 = Array(JRA_fluxes_sensible["var122"]).*area_mask.*land_mask #units: W
ERA5_latentheat_90 = Array(ds_hfds["slhf"])[:,:,1,133:144]./(24*3600).*repeat(permutedims(d_area_ERA5, (2, 1)), 1, 1, 12).*(1 .- Array(ds_lm["lsm"])[:,:,133:144]) #units: W
ERA5_sensibleheat_90 = Array(ds_hfds["sshf"])[:,:,1,133:144]./(24*3600).*repeat(permutedims(d_area_ERA5, (2, 1)), 1, 1, 12).*(1 .- Array(ds_lm["lsm"])[:,:,133:144]) #units: W

In [ ]:
using Oceananigans.Operators: Azᶜᶜᵃ
LH_W = zeros(length(times))  # Ensure the array is 1D with the correct size
SH_W = zeros(length(times))  # Ensure the array is 1D with the correct size
LH_W_coarse = zeros(length(times_coarse))  # Ensure the array is 1D with the correct size
SH_W_coarse = zeros(length(times_coarse))  # Ensure the array is 1D with the correct size

for time_step in eachindex(times)  # Proper way to iterate over indices
    LH_W[time_step] = sum(interior(fluxes["latent_heat"][time_step]*Az))
    SH_W[time_step] = sum(interior(fluxes["sensible_heat"][time_step]*Az))
end

for time_step in eachindex(times_coarse)  # Proper way to iterate over indices
    LH_W_coarse[time_step] = sum(interior(fluxes_coarse["latent_heat"][time_step]*Az))
    SH_W_coarse[time_step] = sum(interior(fluxes_coarse["sensible_heat"][time_step]*Az))
end

In [ ]:
fig = Figure(size = (1200, 800))
ax1 = Axis(fig[1, 1], xlabel="Years", ylabel="Latent Heat [W]", title="Latent Heat")
ax2 = Axis(fig[1, 2], xlabel="Years", ylabel="Sensible Heat [W]", title="Sensible Heat")
ax3 = Axis(fig[2,1], xlabel="Years", ylabel="Temperature", title="Max Surface Temp")

lines!(ax1, range(0, stop=1, length=12), -1 .* sum(ERA5_latentheat_90, dims = (1,2))[1,1,:], color = :blue, label = "ERA5")
lines!(ax2, range(0, stop=1, length=12), -1 .* sum(ERA5_sensibleheat_90, dims = (1,2))[1,1,:], color = :blue)
lines!(ax1, range(0, stop=1, length=12), sum(JRA55_latentheat_90, dims = (1,2))[1,1,:], color = :red, label = "JRA55")
lines!(ax2, range(0, stop=1, length=12), sum(JRA55_sensibleheat_90, dims = (1,2))[1,1,:], color = :red)
lines!(ax1, times/(3600*24*365.25), LH_W, color=:black, label = "ClimaOcean - 1deg Tripolar")
lines!(ax2, times/(3600*24*365.25), SH_W, color=:black)
lines!(ax3, times/(3600*24*365.25), mxtemp, color=:black)
lines!(ax1, times_coarse/(3600*24*365.25), LH_W_coarse, color=:green, label = "ClimaOcean - 5deg LatLonGrid")
lines!(ax2, times_coarse/(3600*24*365.25), SH_W_coarse, color=:green)
lines!(ax3, times_coarse/(3600*24*365.25), mxtemp_coarse, color=:green)
xlims!(ax2, 0,1.5)
xlims!(ax1, 0,1.5)
xlims!(ax3, 0,1.5)
axislegend(ax1, position = :rb)

save("/Users/tsohail/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/uom/ocean-ensembles/figures/flux_issue.png", fig, px_per_unit=3)

display(fig)

## Surface Analysis

In [ ]:
fig = Figure(size = (1200, 800))
# axs = Axis(fig[1, 1], xlabel="Longitude (deg)", ylabel="Latitude (deg)")

time_slice = 3

times = surface_props["T"].times
println(size(times))
title = string("Global 1 degree ocean simulation after ",
                         prettytime(times[time_slice] - times[1]))

axs = Axis(fig[1, 1], xlabel="Longitude (deg)", ylabel="Latitude (deg)")
hm = heatmap!(axs, view(interior(surface_props["T"][time_slice]), :, :, 1), colorrange = (-1,31), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[1, 2], hm, label = "Temperature (ᵒC)")

axs = Axis(fig[2, 1], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, view(interior(surface_props["u"][time_slice]), :, :, 1) , colorrange = (-1,1), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[2, 2], hm, label = "u (m/s)")

axs = Axis(fig[2, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, view(interior(surface_props["v"][time_slice]), :, :, 1) , colorrange = (-1,1), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[2, 4], hm, label = "v (m/s)")

# axs = Axis(fig[2, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
# hm = heatmap!(axs, view(interior(surface_props["w"][time_slice]), :, :, 1), colorrange = (-.1,.1), colormap = :bwr, nan_color=:lightgray)
# Colorbar(fig[2, 4], hm, label = "w (m/s)")

axs = Axis(fig[1, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, view(interior(surface_props["S"][time_slice]), :, :, 1) , colorrange = (33, 38), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[1, 4], hm, label = "Salinity (unitless)")

Label(fig[0, :], title)
save("/Users/tsohail/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/uom/ocean-ensembles/figures/surface_quarter.png", fig, px_per_unit=3)
display(fig)


In [ ]:
fig = Figure(size = (1200, 800))
axs1 = Axis(fig[1, 1], xlabel="Longitude (deg)", ylabel="Latitude (deg)")
hm1 = heatmap!(axs1, view(interior(surface_props["T"][1]), :, :, 1), colorrange=(-1,31), colormap=:bwr, nan_color=:lightgray)
Colorbar(fig[1, 2], hm1, label="Temperature (ᵒC)")

axs2 = Axis(fig[2, 1], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm2 = heatmap!(axs2, view(interior(surface_props["u"][1]), :, :, 1), colorrange=(-1,1), colormap=:bwr, nan_color=:lightgray)
Colorbar(fig[2, 2], hm2, label="u (m/s)")

axs3 = Axis(fig[2, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm3 = heatmap!(axs3, view(interior(surface_props["v"][1]), :, :, 1), colorrange=(-1,1), colormap=:bwr, nan_color=:lightgray)
Colorbar(fig[2, 4], hm3, label="v (m/s)")

# axs4 = Axis(fig[2, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
# hm4 = heatmap!(axs4, view(interior(surface_props["w"][1]), :, :, 1), colorrange=(-.1,.1), colormap=:bwr, nan_color=:lightgray)
# Colorbar(fig[2, 4], hm4, label="w (m/s)")

axs5 = Axis(fig[1, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm5 = heatmap!(axs5, view(interior(surface_props["S"][1]), :, :, 1), colorrange=(33, 38), colormap=:bwr, nan_color=:lightgray)
Colorbar(fig[1, 4], hm5, label="Salinity (unitless)")

title_label = Label(fig[0, :], "Global 1-degree ocean simulation")

# Save animation
record(fig, "/Users/tsohail/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/uom/ocean-ensembles/figures/ocean_surface_simulation_quarter.mp4", 1:length(surface_props["T"].times)) do time_step
    title_label.text = string("Global 1 degree ocean simulation after ", prettytime(surface_props["T"].times[time_step] - surface_props["T"].times[1]))

    hm1[1] = view(interior(surface_props["T"][time_step]), :, :, 1)
    hm2[1] = view(interior(surface_props["u"][time_step]), :, :, 1)
    hm3[1] = view(interior(surface_props["v"][time_step]), :, :, 1)
    # hm4[1] = view(interior(surface_props["w"][time_step]), :, :, 1)
    hm5[1] = view(interior(surface_props["S"][time_step]), :, :, 1)
end


## Zonal analysis

In [ ]:
fig = Figure(size = (800, 800))
# axs = Axis(fig[1, 1], xlabel="Longitude (deg)", ylabel="Latitude (deg)")

time_slice = 715

times = int_za["S_atlantic"].times

S_avg_atl = view(interior(int_za["S_atlantic"][time_slice]/int_za["dV_atlantic"][time_slice]), 1, :, :)
S_avg_ipac = view(interior(int_za["S_pacific"][time_slice]/int_za["dV_pacific"][time_slice]), 1, :, :)
S_avg_glob = view(interior(int_za["S_global"][time_slice]/int_za["dV_global"][time_slice]), 1, :, :)
T_avg_atl = view(interior(int_za["T_atlantic"][time_slice]/int_za["dV_atlantic"][time_slice]), 1, :, :)
T_avg_ipac = view(interior(int_za["T_pacific"][time_slice]/int_za["dV_pacific"][time_slice]), 1, :, :)
T_avg_glob = view(interior(int_za["T_global"][time_slice]/int_za["dV_global"][time_slice]), 1, :, :)

title = string("Global 1 degree ocean simulation after ",
                         prettytime(times[time_slice] - times[1]))

axs = Axis(fig[1, 1], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, T_avg_atl, colorrange = (-1,31), colormap = :deep, nan_color=:lightgray)
Colorbar(fig[1, 2], hm, label = "Temperature (ᵒC)")

axs = Axis(fig[2, 1], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, T_avg_ipac , colorrange = (-1,31), colormap = :deep, nan_color=:lightgray)
Colorbar(fig[2, 2], hm, label = "Temperature (ᵒC)")

axs = Axis(fig[3, 1], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, T_avg_glob , colorrange = (-1,31), colormap = :deep, nan_color=:lightgray)
Colorbar(fig[3, 2], hm, label = "Temperature (ᵒC)")

axs = Axis(fig[1, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, S_avg_atl, colorrange = (34, 37), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[1, 4], hm, label = "Salinity (unitless)")

axs = Axis(fig[2, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, S_avg_ipac , colorrange = (34, 37), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[2, 4], hm, label = "Salinity (unitless)")

axs = Axis(fig[3, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, S_avg_glob , colorrange = (34, 37), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[3, 4], hm, label = "Salinity (unitless)")

Label(fig[0, :], title)

save("/Users/tsohail/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/uom/ocean-ensembles/figures/zonal_average_quarter.png", fig)
display(fig)

In [ ]:
fig = Figure(size = (800, 800))

axs1 = Axis(fig[1, 1], xlabel="Latitude (deg)", ylabel="Depth (m)")
hm1 = heatmap!(axs1, zeros(size(view(interior(int_za["T_atlantic"][1]/int_za["dV_atlantic"][1]), 1, :, :))), 
               colorrange=(-1, 31), colormap=:deep, nan_color=:lightgray)
Colorbar(fig[1, 2], hm1, label="Temperature (ᵒC)")

axs2 = Axis(fig[2, 1], xlabel="Latitude (deg)", ylabel="Depth (m)")
hm2 = heatmap!(axs2, zeros(size(view(interior(int_za["T_pacific"][1]/int_za["dV_pacific"][1]), 1, :, :))), 
               colorrange=(-1, 31), colormap=:deep, nan_color=:lightgray)
Colorbar(fig[2, 2], hm2, label="Temperature (ᵒC)")

axs3 = Axis(fig[3, 1], xlabel="Latitude (deg)", ylabel="Depth (m)")
hm3 = heatmap!(axs3, zeros(size(view(interior(int_za["T_global"][1]/int_za["dV_global"][1]), 1, :, :))), 
               colorrange=(-1, 31), colormap=:deep, nan_color=:lightgray)
Colorbar(fig[3, 2], hm3, label="Temperature (ᵒC)")

axs4 = Axis(fig[1, 3], xlabel="Latitude (deg)", ylabel="Depth (m)")
hm4 = heatmap!(axs4, zeros(size(view(interior(int_za["S_atlantic"][1]/int_za["dV_atlantic"][1]), 1, :, :))), 
               colorrange=(34, 37), colormap=:bwr, nan_color=:lightgray)
Colorbar(fig[1, 4], hm4, label="Salinity (unitless)")

axs5 = Axis(fig[2, 3], xlabel="Latitude (deg)", ylabel="Depth (m)")
hm5 = heatmap!(axs5, zeros(size(view(interior(int_za["S_pacific"][1]/int_za["dV_pacific"][1]), 1, :, :))), 
               colorrange=(34, 37), colormap=:bwr, nan_color=:lightgray)
Colorbar(fig[2, 4], hm5, label="Salinity (unitless)")

axs6 = Axis(fig[3, 3], xlabel="Latitude (deg)", ylabel="Depth (m)")
hm6 = heatmap!(axs6, zeros(size(view(interior(int_za["S_global"][1]/int_za["dV_global"][1]), 1, :, :))), 
               colorrange=(34, 37), colormap=:bwr, nan_color=:lightgray)
Colorbar(fig[3, 4], hm6, label="Salinity (unitless)")

title_label = Label(fig[0, :], "Global 1-degree ocean simulation")

# Save animation
record(fig, "/Users/tsohail/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/uom/ocean-ensembles/figures/zonal_average_quarter.mp4", 1:length(int_za["S_atlantic"].times)) do time_step
    title_label.text = string("Global 1 degree ocean simulation after ", 
                              prettytime(int_za["S_atlantic"].times[time_step] - int_za["S_atlantic"].times[1]))

    hm1[1] = view(interior(int_za["T_atlantic"][time_step]/int_za["dV_atlantic"][time_step]), 1, :, :)
    hm2[1] = view(interior(int_za["T_pacific"][time_step]/int_za["dV_pacific"][time_step]), 1, :, :)
    hm3[1] = view(interior(int_za["T_global"][time_step]/int_za["dV_global"][time_step]), 1, :, :)
    hm4[1] = view(interior(int_za["S_atlantic"][time_step]/int_za["dV_atlantic"][time_step]), 1, :, :)
    hm5[1] = view(interior(int_za["S_pacific"][time_step]/int_za["dV_pacific"][time_step]), 1, :, :)
    hm6[1] = view(interior(int_za["S_global"][time_step]/int_za["dV_global"][time_step]), 1, :, :)
end


In [ ]:
30/1.4583333333333333

In [ ]:
fig = Figure(size = (800, 800))
# axs = Axis(fig[1, 1], xlabel="Longitude (deg)", ylabel="Latitude (deg)")

time_slice = 715

S_avg_ipac_init = view(interior(int_za["S_pacific"][time_slice]/int_za["dV_pacific"][time_slice]), 1, :, :)
S_avg_glob_init = view(interior(int_za["S_global"][time_slice]/int_za["dV_global"][time_slice]), 1, :, :)
T_avg_atl_init = view(interior(int_za["T_atlantic"][time_slice]/int_za["dV_atlantic"][time_slice]), 1, :, :)
T_avg_ipac_init = view(interior(int_za["T_pacific"][time_slice]/int_za["dV_pacific"][time_slice]), 1, :, :)
T_avg_glob_init = view(interior(int_za["T_global"][time_slice]/int_za["dV_global"][time_slice]), 1, :, :)

time_slice = 334

S_avg_atl = view(interior(int_za["S_atlantic"][time_slice]/int_za["dV_atlantic"][time_slice]), 1, :, :)
S_avg_ipac = view(interior(int_za["S_pacific"][time_slice]/int_za["dV_pacific"][time_slice]), 1, :, :)
S_avg_glob = view(interior(int_za["S_global"][time_slice]/int_za["dV_global"][time_slice]), 1, :, :)
T_avg_atl = view(interior(int_za["T_atlantic"][time_slice]/int_za["dV_atlantic"][time_slice]), 1, :, :)
T_avg_ipac = view(interior(int_za["T_pacific"][time_slice]/int_za["dV_pacific"][time_slice]), 1, :, :)
T_avg_glob = view(interior(int_za["T_global"][time_slice]/int_za["dV_global"][time_slice]), 1, :, :)


axs = Axis(fig[1, 1], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, T_avg_atl - T_avg_atl_init, colorrange = (-2,2), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[1, 2], hm, label = "Temperature (ᵒC)")

axs = Axis(fig[2, 1], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, T_avg_ipac - T_avg_ipac_init, colorrange = (-2,2), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[2, 2], hm, label = "Temperature (ᵒC)")

axs = Axis(fig[3, 1], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, T_avg_glob - T_avg_glob_init, colorrange = (-2,2), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[3, 2], hm, label = "Temperature (ᵒC)")

axs = Axis(fig[1, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, S_avg_atl - S_avg_atl_init, colorrange = (-0.5, 0.5), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[1, 4], hm, label = "Salinity (unitless)")

axs = Axis(fig[2, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, S_avg_ipac - S_avg_ipac_init, colorrange = (-0.5, 0.5), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[2, 4], hm, label = "Salinity (unitless)")

axs = Axis(fig[3, 3], xlabel="Longitude (deg)", ylabel="Depth (m)")
hm = heatmap!(axs, S_avg_glob - S_avg_glob_init, colorrange = (-0.5, 0.5), colormap = :bwr, nan_color=:lightgray)
Colorbar(fig[3, 4], hm, label = "Salinity (unitless)")

Label(fig[0, :], title)
save("/Users/tsohail/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/uom/ocean-ensembles/figures/zonal_anomaly_quarter.png", fig)

display(fig)

## Global

In [ ]:
grid = RectilinearGrid(size=(4, 5, 6), extent=(2, 3, 4))
c = CenterField(grid)
Integral(Field(Integral(c, dims=(2, 3))), dims=(1))

In [ ]:
int_za["T_global"][1]

In [ ]:
using Oceananigans.Fields: instantiate
using Oceananigans: location
using ClimaOcean


In [ ]:

Nx, Ny, Nz = 60, 30, 20

# grid = Oceananigans.OrthogonalSphericalShellGrids.TripolarGrid(CPU(); size=(Nx, Ny, Nz),
#                                                                           halo=(7, 7, 7),
#                                                                             z=(-1000, 0))

grid = LatitudeLongitudeGrid(CPU();
                             size = (Nx, Ny, Nz),
                             halo = (7, 7, 7),
                             z = (-1000, 0),
                             latitude  = (-75, 75),
                             longitude = (0, 360))

ocean = ocean_simulation(grid)


In [ ]:
using StaticArrays
using PolygonOps
using Oceananigans.Fields: instantiate, location

function basin_mask(grid::Union{TripolarGrid, ImmersedBoundaryGrid{<:Any, <:Any, <:Any, <:Any, <:TripolarGrid}, LatitudeLongitudeGrid}, 
    basin::AbstractString, 
    var::Oceananigans.Field)

    IndLonsPts=[20, 20, 40,100, 100, 110,145,145,20];
    IndLatsPts=[-90, 28, 30, 30, 0, -10,-10,-90,-90];
    
    PacLonsPts=[145, 145, 110,100, 100, 260,260,300,300, 145];
    PacLatsPts=[-90, -10, -10, 0, 90, 90,20,0,-90,-90];
    
    # Atlantic is a bit more complicated
    AtleastLonsPts = [260,260,300,300, 360, 360, 260];
    AtleastLatsPts = [90,20,0,-90,-90, 90,90] ;
    AtlwestLonsPts = [0,20,20,0,0]
    AtlwestLatsPts = [-90,-90,28,30,-90] ;
    AtlarcticLonsPts = [0,20,20,0,0]
    AtlarcticLatsPts = [50,55,90,90,50]

    Indpolygon = SVector.(IndLonsPts, IndLatsPts)    # boundary of the polygon
    Pacpolygon = SVector.(PacLonsPts, PacLatsPts)    # boundary of the polygon
    # Atlantic has multiple polygons due to lon grid bw 0 and 360
    Atleastpolygon = SVector.(AtleastLonsPts, AtleastLatsPts)    # boundary of the polygon
    Atlwestpolygon = SVector.(AtlwestLonsPts, AtlwestLatsPts)    # boundary of the polygon
    Atlarcticpolygon = SVector.(AtlarcticLonsPts, AtlarcticLatsPts)    # boundary of the polygon

    lons = λnodes(grid, instantiate.(location(var))...)
    lats = φnodes(grid, instantiate.(location(var))...)

    if isa(grid, LatitudeLongitudeGrid)
        X,Y = meshgrid(lons[1:Nx], lats[1:Ny])
        points = vec(SVector.(X, Y))
        lons, lats = X,Y
    else
        points = vec(SVector.(lons,lats))
    end
    
    if basin in ["indian", "Indian"]
        polygon = Indpolygon
        mask = transpose(reshape([inpolygon(p, polygon; in=true, on=false, out=false) for p in points], size(lats)))

    elseif basin in ["pacific", "Pacific"]
        polygon = Pacpolygon
        mask = transpose(reshape([inpolygon(p, polygon; in=true, on=false, out=false) for p in points], size(lats)))

    elseif basin in ["atlantic", "Atlantic"]
        polygon = Atleastpolygon
        mask_1 = transpose(reshape([inpolygon(p, polygon; in=true, on=false, out=false) for p in points], size(lats)))
        
        polygon = Atlwestpolygon
        mask_2 = transpose(reshape([inpolygon(p, polygon; in=true, on=false, out=false) for p in points], size(lats)))
        
        polygon = Atlarcticpolygon
        mask_3 = transpose(reshape([inpolygon(p, polygon; in=true, on=false, out=false) for p in points], size(lats)))
    
        mask = mask_1 .+ mask_3 .+ mask_2 
    
    elseif basin in ["indo-pacific", "Indo-pacific", "Indo-Pacific"]
        polygon = Indpolygon
        mask_1 = transpose(reshape([inpolygon(p, polygon; in=true, on=false, out=false) for p in points], size(lats)))
        
        polygon = Pacpolygon
        mask_2 = transpose(reshape([inpolygon(p, polygon; in=true, on=false, out=false) for p in points], size(lats)))
    
        mask = mask_1 .+ mask_2  
    else
        throw("Basin unknown, must be one of Indian, Atlantic, Pacific, or Indo-Pacific")
    end

    is_valid = maximum(mask) == 1
    is_valid || throw(ErrorException("Maximum value of mask is not 1"))    
    bool_mask = convert(Array{Bool}, mask)
    return bool_mask
end

In [ ]:
mask_atl = basin_mask(grid, "Atlantic", ocean.model.velocities[1])

In [ ]:
ocean.model.velocities.u

avg_u = compute!(Average(ocean.model.velocities.u, condition=mask_atl))
display(avg_u)

In [ ]:
fig = heatmap!(lon, lat, mask_Ipac_u)
display(fig)

In [ ]:
println(size(tplats[1:Nx, 1:Ny]))
println(size(tplons[1:Nx, 1:Ny]))
println(size(transpose(mask_Ipac_u)))

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1], xlabel="Longitude", ylabel="Latitude", title="Basin Map")
hm = heatmap!(ax, lon, lat, surface_props["S"][1], colormap=:viridis)
hm = heatmap!(ax, lon, lat, (mask_Ipac_u), colormap=:bwr, alpha = 0.5)

# CairoMakie.lines!(Indpolygon)
# CairoMakie.lines!(Pacpolygon)
# CairoMakie.lines!(Atleastpolygon)
# CairoMakie.lines!(Atlwestpolygon)
# CairoMakie.lines!(Atlarcticpolygon)

Colorbar(fig[1, 2], hm)
display(fig)

In [ ]:
using PolygonOps, StaticArrays

IndLonsPts=[20, 20, 40,100, 100, 110,145,145,20];
IndLatsPts=[-90, 28, 30, 30, 0, -10,-10,-90,-90];

PacLonsPts=[145-0.1, 145-0.1, 110-0.1,100-0.1, 100-0.1, 260,260,300,300, 145];
PacLatsPts=[-90, -10, -10, 0, 90, 90,20,0,-90,-90];

#Atlantic is a bit more complicated
AtleastLonsPts = [260,260,300,300, 360, 360, 360];
AtleastLatsPts = [90,20,0,-90,-90, 90,90] ;
AtlwestLonsPts = [0,20,20,0,0]
AtlwestLatsPts = [-90,-90,28,30,-90] ;
AtlarcticLonsPts = [0,20,20,0,0]
AtlarcticLatsPts = [50,55,90,90,50]

Indpolygon = SVector.(IndLonsPts, IndLatsPts)    # boundary of the polygon
Pacpolygon = SVector.(PacLonsPts, PacLatsPts)    # boundary of the polygon
Atleastpolygon = SVector.(AtleastLonsPts, AtleastLatsPts)    # boundary of the polygon
Atlwestpolygon = SVector.(AtlwestLonsPts, AtlwestLatsPts)    # boundary of the polygon
Atlarcticpolygon = SVector.(AtlarcticLonsPts, AtlarcticLatsPts)    # boundary of the polygon

lon  = 0:1:360;   # 1440 values
lat  = -90:1:90;
X, Y = meshgrid(lon, lat)   #  720 values
points = vec(SVector.(X,Y))

mask = transpose(reshape([inpolygon(p, Indpolygon; in=true, on=false, out=false) for p in points], size(Y)))



In [ ]:
Atlantic_mask

In [ ]:
lon = -16.25-180:2.5:376.25-180
lat = -90:1:90

basins = [(whichshortlistoceans(l, φ, OCEANS)) for φ in lon, l in lat]

size(basins)
unique_basins = unique(vcat(basins...))
basin_map = Dict(b => i for (i, b) in enumerate(unique_basins))

boolean_basins = similar(basins, Int)

for φ in 1:length(lat)
    for l in 1:length(lon)
        if isempty(basins[l, φ])
            boolean_basins[l, φ] =  0
        else
            # Otherwise, map the basin to its corresponding index in the unique_basins list
            basin_name = basins[l, φ][1]
            # Check if the basin exists in the basin_map and assign the corresponding index
            if haskey(basin_map, basin_name)
                @info "I am here"
                boolean_basins[l, φ] = basin_map[basin_name]
            else
                boolean_basins[l, φ] = 0  # If basin name doesn't exist in basin_map, assign 0
            end
        end
    end
end

display(boolean_basins)

fig = Figure()
ax = Axis(fig[1, 1], xlabel="Longitude", ylabel="Latitude", title="Basin Map")
hm = heatmap!(ax, lon, lat, boolean_basins)
Colorbar(fig[1, 2], hm)
display(fig)

In [ ]:
# Example polygon (e.g., a simple triangle)
polygon_lat = [40.0, 41.0, 42.0, 40.0]  # latitudes
polygon_lon = [-75.0, -74.0, -75.0, -75.0]  # longitudes


In [ ]:
whichshortlistoceans(60, -100, OCEANS)

In [ ]:
fig = Figure(size = (800, 800))

length(keys(int_za))

for keys in keys(int_za)
axTavg = Axis(fig[1, 1], xlabel="Time (days)", ylabel="Temperature (∘C)")
axSavg = Axis(fig[2, 1], xlabel="Time (days)", ylabel="Salinity (g/kg)")
axTint = Axis(fig[1, 2], xlabel="Time (days)", ylabel="Temperature (∘C)")
axSint = Axis(fig[2, 2], xlabel="Time (days)", ylabel="Salinity (g/kg)")

lines!(axTavg, Tavg.times/(3600*24), (view(Tavg, 1, 1, 1, :)))
lines!(axSavg, Savg.times/(3600*24), (view(Savg, 1, 1, 1, :)))
heatmap!(axTint, Tint.grid.z.cᵃᵃᶜ, Tint.times/(3600*24),  (view(interior(Tint), 1, 1, :, :)))
heatmap!(axSint, Sint.grid.z.cᵃᵃᶜ, Sint.times/(3600*24), (view(interior(Sint), 1, 1, :, :)))

display(fig)

In [ ]:
fig = Figure(size = (800, 800))
axvolint = Axis(fig[1, 1], xlabel="Time (days)", ylabel="volume (m3)")
lines!(axvolint, Vint.times/(3600*24), (view(Vint, 1, 1, 1, :)))

display(fig)


In [ ]:
fig = Figure(size = (800, 800))
axs = Axis(fig[1, 1], xlabel="Longitude (deg)", ylabel="Latitude (deg)")

heatmap!(axs, view(interior(T_surf_1_4_deg[1]), :, :, 1))
display(fig)